### Check the machine

In [ ]:
! nvidia-smi

Thu Oct 20 05:16:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Setup

In [ ]:
#@title install dependencies
! git clone https://github.com/ashawkey/stable-dreamfusion.git

%cd stable-dreamfusion

# install requirements
! pip install -r requirements.txt
! pip install git+https://github.com/NVlabs/nvdiffrast/

# install extension modules
! bash scripts/install_ext.sh

Cloning into 'stable-dreamfusion'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 224 (delta 45), reused 44 (delta 30), pack-reused 153
Receiving objects: 100% (224/224), 125.11 KiB | 12.51 MiB/s, done.
Resolving deltas: 100% (120/120), done.
/content/stable-dreamfusion
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 120 kB 38.0 MB/s 
     |████████████████████████████████| 659 kB 60.4 MB/s 
     |████████████████████████████████| 125 kB 72.6 MB/s 
     |████████████████████████████████| 265 kB 69.2 MB/s 
     |████████████████████████████████| 237 kB 76.3 MB/s 
     |████████████████████████████████| 2.8 MB 50.9 MB/s 
     |████████████████████████████████| 163 kB 75.1 MB/s 
     |████████████████████████████████| 255 kB 62.4 MB/s 
     |████████████████████████████████| 5.3 MB 55.1 MB/s 
    

In [ ]:
#@title login to huggingface to download stable diffusion
from huggingface_hub import notebook_login
from google.colab import output

output.enable_custom_widget_manager()
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


### Training & Testing
* The training already includes exporting video and mesh once finished. So you don't need to run the testing if you wait until the training finishes. But if you interrupt the training, a manual testing is needed.
* It takes about 0.7s per training step, so the default 5000 training steps take around 1 hour to finish. A larger `Training_iters` usually leads to better results.
* Increasing `Training_nerf_resolution` leads to better rendering quality too, but requires more GPU memory.
* If the NeRF fails to learn anything (empty scene, only background), try to decrease `Lambda_entropy` which regularizes the learned opacity.

In [ ]:
#@markdown ####**Training Settings:**
Prompt_text = "photo of a delicious hamburger" #@param {type: 'string'}
Training_iters = 50 #@param {type: 'integer'}
Learning_rate = 1e-3 #@param {type: 'number'}
Training_nerf_resolution = 64  #@param {type: 'integer'}
# CUDA_ray = True #@param {type: 'boolean'}
# View_dependent_prompt = True #@param {type: 'boolean'}
# FP16 = True #@param {type: 'boolean'}
Seed = 0 #@param {type: 'integer'}
Lambda_entropy = 1e-4 #@param {type: 'number'}
Checkpoint = 'scratch' #@param {type: 'string'}

#@markdown ---

#@markdown ####**Output Settings:**
Workspace = "trial" #@param{type: 'string'}
# Save_mesh = True #@param {type: 'boolean'}

# processings
Prompt_text = "'" + Prompt_text + "'"

In [ ]:
#@title start training
%cd stable-dreamfusion
%run main.py -O --text {Prompt_text} --workspace {Workspace} --iters {Training_iters} --lr {Learning_rate} --w {Training_nerf_resolution} --h {Training_nerf_resolution} --seed {Seed} --lambda_entropy {Lambda_entropy} --ckpt {Checkpoint} --save_mesh

/content/stable-dreamfusion


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Namespace(H=800, O=True, O2=False, W=800, albedo_iters=1000, angle_front=60, angle_overhead=30, backbone='grid', bg_radius=1.4, bound=1, ckpt='scratch', cuda_ray=True, density_thresh=10, dir_text=True, dt_gamma=0, eval_interval=10, fovy=60, fovy_range=[40, 70], fp16=True, gui=False, guidance='stable-diffusion', h=64, iters=50, jitter_pose=False, lambda_entropy=0.0001, lambda_opacity=0, lambda_orient=0.01, light_phi=0, light_theta=60, lr=0.001, max_ray_batch=4096, max_spp=1, max_steps=1024, min_near=0.1, num_steps=64, radius=3, radius_range=[1.0, 1.5], save_mesh=True, seed=0, test=False, text='photo of a delicious hamburger', update_extra_interval=16, upsample_steps=64, w=64, workspace='trial')
NeRFNetwork(
  (encoder): GridEncoder: input_dim=3 num_levels=16 level_dim=2 resolution=16 -> 2048 per_level_scale=1.3819 params=(6119864, 2) gridtype=tiled align_corners=False
  (sigma_net): MLP(
    (net): ModuleList(
      (0): Linear(in_features=32, out_features=64, bias=True)
      (1): Line

0it [00:00, ?it/s]

[INFO] try to load hugging face access token from the default place, make sure you have run `huggingface-cli login`.
[INFO] loading stable diffusion...
[INFO] loaded stable diffusion!


...

==> Finished Test.

100% 100/100 [00:49<00:00,  2.03it/s]


==> Saving mesh to trial/mesh

[INFO] running xatlas to unwrap UVs for mesh: v=(68848, 3) f=(137692, 3)
[INFO] writing obj mesh to trial/mesh/mesh.obj
[INFO] writing vertices (68848, 3)
[INFO] writing vertices texture coords (100646, 2)
[INFO] writing faces (137692, 3)


==> Finished saving mesh.

In [4]:
#@markdown ####**Testing Settings:**

Workspace_test = "trial" #@param{type: 'string'}
# Save_mesh = True #@param {type: 'boolean'}

In [5]:
#@title (optional) testing 
%run main.py -O --test --workspace {Workspace_test} --save_mesh

Namespace(H=800, O=True, O2=False, W=800, albedo_iters=1000, angle_front=60, angle_overhead=30, backbone='grid', bg_radius=1.4, bound=1, ckpt='latest', cuda_ray=True, density_thresh=10, dir_text=True, dt_gamma=0, eval_interval=10, fovy=60, fovy_range=[40, 70], fp16=True, gui=False, guidance='stable-diffusion', h=64, iters=10000, jitter_pose=False, lambda_entropy=0.0001, lambda_opacity=0, lambda_orient=0.01, light_phi=0, light_theta=60, lr=0.001, max_ray_batch=4096, max_spp=1, max_steps=1024, min_near=0.1, num_steps=64, radius=3, radius_range=[1.0, 1.5], save_mesh=True, seed=0, test=True, text=None, update_extra_interval=16, upsample_steps=64, w=64, workspace='trial')
NeRFNetwork(
  (encoder): GridEncoder: input_dim=3 num_levels=16 level_dim=2 resolution=16 -> 2048 per_level_scale=1.3819 params=(6119864, 2) gridtype=tiled align_corners=False
  (sigma_net): MLP(
    (net): ModuleList(
      (0): Linear(in_features=32, out_features=64, bias=True)
      (1): Linear(in_features=64, out_feat

[INFO] Trainer: df | 2022-10-20_05-57-25 | cuda | fp16 | trial

[INFO] #parameters: 12249015

[INFO] Loading latest checkpoint ...

[INFO] Latest checkpoint is trial/checkpoints/df_ep0011.pth

...

==> Finished Test.

100% 100/100 [00:33<00:00,  3.02it/s]


==> Saving mesh to trial/mesh

[INFO] running xatlas to unwrap UVs for mesh: v=(157106, 3) f=(309442, 3)
[INFO] writing obj mesh to trial/mesh/mesh.obj
[INFO] writing vertices (157106, 3)
[INFO] writing vertices texture coords (294144, 2)
[INFO] writing faces (309442, 3)


==> Finished saving mesh.

### Display results
* RGB and Depth video are located at `{Workspace}/results/*.mp4`
* Mesh is under `{Workspace}/mesh/`, you could see three files named `mesh.obj`, `mesh.mtl`, and `albedo.png`.

In [ ]:
#@title display RGB video
import os
import glob
from IPython.display import HTML
from base64 import b64encode

def get_latest_file(path):
  dir_list = glob.glob(path)
  dir_list.sort(key=lambda x: os.path.getmtime(x))
  return dir_list[-1]

def show_video(video_path, video_width = 600):
   
  video_file = open(video_path, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"

  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")
 
rgb_video = get_latest_file(os.path.join(Workspace, 'results', '*_rgb.mp4'))
show_video(rgb_video)